In [21]:
import pandas as pd

from os.path import join

from collections import Counter

import re

from datetime import datetime

In [2]:
basePath = join("..", "data", "cleaned")

fileNames = [
    "Swiggy 01-01-2025 - 15-04-2025",
    "Swiggy 09-04-2025 - 22-06-2025",
    "Swiggy 22-06-2025 - 10-07-2025",
]

In [3]:
df = [pd.read_csv(join(basePath, f"{fileName}.csv")) for fileName in fileNames]

In [ ]:
masterDf = pd.DataFrame()
for extractInvoiceVersionLocation in df:
    masterDf = pd.concat([masterDf, extractInvoiceVersionLocation], ignore_index=True)
masterDf["Invoice Version"] = 1

In [ ]:
def extractInvoiceVersionLocation(row: pd.Series):
    locations = [
        "Nandanvan",
        "Dharampeth",
        "Mahal",
        "Ayodhya Nagar",
        "Sai Mandir",
        "Manish Nagar",
        "Byramji",
    ]

    chances = [0] * len(locations)
    invoiceVersion = 1

    for i, location in enumerate(locations):
        counterOriginal = Counter(row["Location"])
        counterLocation = Counter(location)
        chances[i] = (
            sum((counterOriginal & counterLocation).values())
            * 2
            / (sum(counterOriginal.values()) + sum(counterLocation.values()))
        )

    numbers = re.findall(r"\d+(?:\.\d+)?", row["Location"])
    if numbers:
        invoiceVersion = int(numbers[0])
    row["Location"] = locations[chances.index(max(chances))]
    row["Invoice Version"] = invoiceVersion

    return row


masterDf = masterDf.apply(extractInvoiceVersionLocation, axis=1)

,Article Code,Item Description,UoM,Dispatched Qty,Rate,Total Amount,Date,Location,Invoice Version
0,1013,Beetroot,500g,9,13.0,117.0,2025-04-15,Nandanvan,1
1,7043,Brinjal Nagpur,250g,2,12.0,24.0,2025-04-15,Nandanvan,1
2,1037,Cauliflower,1piece,35,22.0,770.0,2025-04-15,Nandanvan,1
3,1049,Colocasia (Arbi),250g,1,21.0,21.0,2025-04-15,Nandanvan,1
4,2609,Cowpea Beans,250g,8,18.0,144.0,2025-04-15,Nandanvan,1
...,...,...,...,...,...,...,...,...,...
22461,2116,Onion,1kg,36,30.0,1080.0,2025-06-22,Sai Mandir,1
22462,4068,Pumpkin Cut,200g,2,20.0,40.0,2025-06-22,Sai Mandir,1
22463,3967,Purple Bharta Brinjal,1piece,7,25.0,175.0,2025-06-22,Sai Mandir,1
22464,1113,Ridge Gourd,2pieces,2,30.0,60.0,2025-06-22,Sai Mandir,1


In [ ]:
masterDf["Date"] = masterDf["Date"].apply(
    lambda date: datetime.strptime(date, "%Y-%m-%d").strftime("%m-%d-%Y")
)

In [24]:
masterDf.to_csv(join(basePath, "Swiggy 01-01-2025 - 10-07-2025.csv"), index=False)